In [7]:
import pandas as pd
import numpy as np

In [8]:
df_beijing = pd.read_csv("data/Beijing_labeled.csv")
df_shenyang = pd.read_csv("data/Shenyang_labeled.csv")
df_train = pd.concat([df_beijing, df_shenyang], axis=0)
df_train


,season,DEWP,HUMI,PRES,TEMP,Iws,precipitation,cbwd_NE,cbwd_NW,cbwd_SE,PM_HIGH
0,4,-8.0,79.00,1026.0,-5.0,23.69,0.0,0,0,1,1.0
1,4,-11.0,85.00,1021.0,-9.0,105.93,1.1,0,0,1,0.0
2,4,-21.0,43.00,1030.0,-11.0,117.55,0.0,0,1,0,0.0
3,4,-25.0,33.00,1034.0,-12.0,39.35,0.0,1,0,0,0.0
4,4,-24.0,30.00,1034.0,-10.0,59.00,0.0,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
819,4,-22.0,50.63,1038.0,-14.0,4.00,0.0,0,0,0,1.0
820,4,-16.0,52.47,1034.0,-8.0,4.00,0.0,0,0,0,1.0
821,4,-13.0,45.91,1028.0,-3.0,40.00,0.0,0,0,1,1.0
822,4,-4.0,80.04,1029.0,-1.0,6.00,0.0,0,1,0,1.0


In [14]:
df_guangzhou = pd.read_csv("data/Guangzhou_labeled.csv")
df_shanghai = pd.read_csv("data/Shanghai_labeled.csv")
df_test = pd.concat([df_guangzhou, df_shanghai], axis=0)
df_test


,season,DEWP,HUMI,PRES,TEMP,Iws,precipitation,cbwd_NE,cbwd_NW,cbwd_SE,PM_HIGH
0,3.0,15.2,62.00,1013.9,22.9,7.3,0.0,1,0,0,0.0
1,3.0,10.7,43.00,1013.7,24.0,5.2,0.0,1,0,0,0.0
2,3.0,8.8,42.00,1014.4,22.3,9.2,0.0,1,0,0,0.0
3,3.0,12.1,51.00,1013.2,22.7,9.5,0.0,1,0,0,0.0
4,3.0,15.3,76.00,1011.5,19.6,3.0,0.0,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1346,4.0,7.0,71.47,1029.0,12.0,22.0,0.0,1,0,0,0.0
1347,4.0,-2.0,49.22,1036.0,8.0,5.0,0.0,1,0,0,0.0
1348,4.0,0.0,49.80,1033.0,10.0,2.0,0.0,0,0,0,0.0
1349,4.0,4.0,58.03,1029.0,12.0,6.0,0.0,0,1,0,0.0


In [25]:
X_train = df_train.iloc[::, :-1].to_numpy()  # features
Y_train = df_train.iloc[::, -1].to_numpy()  # labels

X_test = df_test.iloc[::, :-1].to_numpy()  # features
Y_test = df_test.iloc[::, -1].to_numpy()  # labels


In [41]:
import numpy as np


class KNN:
    def __init__(self, X: np.ndarray, Y: np.ndarray, k: int = 10):
        self.X = X
        self.Y = Y
        self.k = k

    def predict(self, x: np.ndarray) -> int:
        self.neighbors = np.zeros((self.X.shape[0], 3))

        for idx, row in enumerate(self.X):
            dist = self._euclid_distance(x, row)
            item_class = self.Y[idx]
            self.neighbors = self._update_neighbors(
                self.neighbors, item_class, dist, idx
            )

        # sort calculated neighbors by distance
        self.neighbors = self.neighbors[self.neighbors[:, 0].argsort()][::-1]
        prediction = self._pick_max()
        return prediction

    def score(self, X: np.ndarray, Y: np.ndarray) -> float:
        # make predictions for test set
        predictions = np.zeros(X.shape[0])
        for idx, x in enumerate(X):
            predictions[idx] = self.predict(x)

        # calc accuracy
        accuracy = np.sum(np.equal(predictions, Y)) / Y.shape[0]
        return accuracy

    def _pick_max(self) -> int:
        k_neighbors = self.neighbors[: self.k, ::]
        cnt_zero = np.count_nonzero(k_neighbors == 0)
        cnt_one = np.count_nonzero(k_neighbors == 1)
        return 0 if cnt_zero > cnt_one else 1

    def _update_neighbors(
        self, neighbors: np.ndarray, item_class: int, dist: float, idx: int
    ) -> np.ndarray:
        # add new neighbor to array and then sort it
        neighbors[idx][0], neighbors[idx][1], neighbors[idx][2] = dist, item_class, idx
        return neighbors

    def _euclid_distance(self, x: np.ndarray, y: np.ndarray) -> float:
        dist = np.linalg.norm(x - y)
        return dist


In [42]:
knn = KNN(X_train, Y_train)
x = X_test[10]
pred = knn.predict(x)
print(f"Prediction for tenth row of testing set: {pred}")


In [43]:
acc = knn.score(X_test, Y_test)
print(f"Accuracy of model: {round(acc*100, 2)}")


0.9119496855345912